In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('F:/thesis/temp/videos_face_embeddings_1fps.csv')

# Create a dictionary to store counters for each unique link
counter_dict = {}

# Iterate through the DataFrame and update the new column based on the conditions
new_column_values = []
for link in df['Link']:
    if link in counter_dict:
        counter_dict[link] += 1
    else:
        counter_dict[link] = 1
    new_column_values.append(counter_dict[link])

# Add the new column to the DataFrame
df.insert(1, 'New Column', new_column_values)

# Save the updated DataFrame to a new CSV file
df.to_csv('F:/thesis/temp/updated_file_1fps.csv', index=False)

print("CSV file updated successfully!")


### Remove if greater than 15

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('F:/thesis/temp/updated_file_1fps.csv')

# Filter out rows where 'New Column' value is greater than 15
df = df[df['New Column'] <= 15]

# Save the filtered DataFrame to a new CSV file
df.to_csv('F:/thesis/temp/filtered_file.csv', index=False)

print("Filtered CSV file saved successfully!")


### Add counter column

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('F:/thesis/Features/Final/Updated/Video_features_final_1.csv')

# Initialize counter variable
counter = 1

# Check if the current link is different from the previous one, and increment counter accordingly
df['Counter'] = (df['Link'] != df['Link'].shift()).cumsum()

# Save the updated DataFrame to a new CSV file
df.to_csv('F:/thesis/Features/Final/Updated/Video_features_final.csv', index=False)

print("CSV file updated successfully!")


In [ ]:
import pandas as pd

# Load the first CSV file
df1 = pd.read_csv('F:/thesis/Data/Final//audio_features_final_1.csv')
# Load the second CSV file
df2 = pd.read_csv('F:/thesis/Data/Final//MFCC_Features_1024.csv')
# Extract only the file names (without extension) from the links
df1['Link_Name'] = df1['Link'].str.split('\\').str[-1].str[:-4]
df2['Link_Name'] = df2['Link'].str.split('\\').str[-1].str[:-4]

# Get the set of unique link names from the first CSV file
link_names_in_first = set(df1['Link_Name'])

# Filter out rows from the second CSV file where the link name does not exist in the first CSV file
df2 = df2[df2['Link_Name'].isin(link_names_in_first)]

# Save the updated DataFrame to a new CSV file
df2.to_csv('F:/thesis/Data/Final/updated_second_file.csv', index=False)

print("CSV file updated successfully!")


# Conversion from Vidoes to Audio

In [ ]:
import os
from moviepy.editor import *

# Directory containing video files
video_directory = 'F:/thesis/Data/F6/F71'

# Directory where you want to save audio files
output_directory = 'F:/thesis/Data/F6/F71/Audio'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Get a list of all video files in the input directory
video_files = [file for file in os.listdir(video_directory) if file.endswith(".mp4")]

for video_file in video_files:
    # Construct the full path of the input video
    video_path = os.path.join(video_directory, video_file)

    # Load the video file
    video_clip = VideoFileClip(video_path)

    # Extract audio from the video
    audio_clip = video_clip.audio

    # Specify the output audio file path
    output_audio_path = os.path.join(output_directory, f"{os.path.splitext(video_file)[0]}.mp3")

    # Write the audio to the output file
    audio_clip.write_audiofile(output_audio_path)

    # Close the video and audio clips
    video_clip.close()
    audio_clip.close()

    print(f"Audio extracted and saved as {output_audio_path}")


# Converion from audio to text

In [ ]:
import os
import whisper
import csv

# Initialize the Whisper ASR recognizer with the Urdu model
model = whisper.load_model('large')

# Directory containing your MP3 audio files (use absolute or relative path)
audio_directory = r"F:/thesis"

# Create a CSV file to store the transcriptions
csv_file = "transcriptions.csv"

# Initialize a list to store the transcription data
transcription_data = []

# Process each audio file in the directory
for filename in os.listdir(audio_directory):
    if filename.endswith(".mp3"):
        audio_file = os.path.join(audio_directory, filename)

        # Check if the audio file exists
        if os.path.exists(audio_file):
            print(f"Processing {audio_file}...")

            try:
                # Perform speech recognition on the audio file
                transcription = model.transcribe(audio_file)

                # Print the transcribed text
                print("Transcription (Urdu):")
                print(transcription)
                print("\n" + "=" * 40 + "\n")

                # Append the transcription and audio file link to the data list
                transcription_data.append([audio_file, transcription])
            except Exception as e:
                print(f"Error processing {audio_file}: {str(e)}")
        else:
            print(f"Audio file not found: {audio_file}")

# Write the transcription data to a CSV file
with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["Audio Link", "Transcription"])
    csv_writer.writerows(transcription_data)

print(f"Transcriptions saved to {csv_file}")


# Mean of video Features

In [ ]:
import pandas as pd

# Read the data from the CSV file
data = pd.read_csv('F:/videos_features_F6.csv')

# List of columns to consider for calculating the mean (all columns except the 'Link' and 'Label' columns)
columns_to_mean = data.columns.difference(['Link', 'Label'])

# Filter non-numeric values
data[columns_to_mean] = data[columns_to_mean].apply(pd.to_numeric, errors='coerce')

# Calculate the mean for each group of identical 'Link' values
averages = data.groupby('Link', sort=False)[columns_to_mean].mean().reset_index()

# Add the 'Label' column back to the result
averages = averages.merge(data[['Link', 'Label']], on='Link', how='left')

# Reorder the columns to have 'Link' as the first column
averages = averages[['Link', 'Label'] + list(columns_to_mean)]

# Keep only the first row for each unique 'Link' value, preserving the original order
averages = averages.drop_duplicates(subset='Link', keep='first')

# Save the result to a new CSV file
averages.to_csv('videos_features_F6_averages.csv', index=False)


# Remove those record which are not common in all files

In [12]:
import pandas as pd

# Load the first CSV file into a DataFrame
df2 = pd.read_csv('F:/thesis/Features/Final/Updated/Urdu_embeddings.csv') 

# Load the second CSV file into another DataFrame
df1 = pd.read_csv('F:/thesis/Features/Final/Updated/Audio_Features_final_wave2vec.csv') 
#input_excel_file = "F:/thesis/Data/F6/Text/urdu_embeddings_xlmr.csv"
#df1 = pd.read_excel(input_excel_file)

# Split the 'link' column on "/" and get the last part
df1['last_part'] = df1['Link'].str.split('\\').str[-1].str[:-4]
df2['last_part'] = df2['Link'].str.split('\\').str[-1].str[:-4]

#print(df1['last_part'][1])

# Create a new DataFrame containing only the rows with last parts present in both files
merged_df = df2[df2['last_part'].isin(df1['last_part'])]

#Save the merged DataFrame to a new CSV file
#merged_df.to_csv('A_Updated.csv', index=False)

output_file = 'F:/thesis/Features/Final/Updated/transcriptions_up.csv'
merged_df.to_csv(output_file, index=False)
#merged_df.to_excel(output_file, index=False)


# Remove those records which are empty

In [ ]:
import pandas as pd

# Replace 'your_file.xlsx' with the actual file path
file_path = 'F:/transcriptions_F6.xlsx'

# Replace 'Text' with the name of the column where you want to remove empty cells
column_name = 'Transcription'

# Read the Excel file into a pandas DataFrame
df = pd.read_excel(file_path)

# Drop rows with empty cells in the specified column
df = df.dropna(subset=[column_name])

# Save the updated DataFrame to a new Excel file
updated_file_path = 'transcriptions_F6_final.xlsx'  # Replace with your desired file path
df.to_excel(updated_file_path, index=False)

print("Empty cells removed and the updated data is saved to", updated_file_path)


In [ ]:
import os
import torch
from moviepy.editor import VideoFileClip

folder_path = 'E:/Study/VidCropper v 2.2/VidCropper/cropped/temp'

# Function to calculate total time in seconds
def calculate_total_time(folder_path):
    total_time = 0
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4"):
            video_path = os.path.join(folder_path, filename)
            video_clip = VideoFileClip(video_path)
            # Get video duration in seconds
            video_duration = video_clip.duration
            total_time += video_duration
            video_clip.close()
    return total_time

# Use CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Calculate total time
total_time_seconds = calculate_total_time(folder_path)

# Convert total time to hours, minutes, and seconds
total_hours = int(total_time_seconds // 3600)
total_minutes = int((total_time_seconds % 3600) // 60)
total_seconds = int(total_time_seconds % 60)

print(f"Total time: {total_hours} hours, {total_minutes} minutes, {total_seconds} seconds")


In [ ]:
import os
import torch
from moviepy.editor import VideoFileClip

folder_path = 'E:/Study/VidCropper v 2.2/VidCropper/cropped/Happy'

# Function to calculate total time in seconds
def calculate_total_time(folder_path):
    total_time = 0
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4"):
            video_path = os.path.join(folder_path, filename)
            video_clip = VideoFileClip(video_path)
            # Get video duration in seconds
            video_duration = video_clip.duration
            total_time += video_duration
            video_clip.close()
    return total_time

# Use CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Calculate total time
total_time_seconds = calculate_total_time(folder_path)

# Convert total time to hours, minutes, and seconds
total_hours = int(total_time_seconds // 3600)
total_minutes = int((total_time_seconds % 3600) // 60)
total_seconds = int(total_time_seconds % 60)

print(f"Total time: {total_hours} hours, {total_minutes} minutes, {total_seconds} seconds")


In [ ]:
import os
import torch
from moviepy.editor import VideoFileClip

folder_path = 'E:/Study/VidCropper v 2.2/VidCropper/cropped/Love'

# Function to calculate total time in seconds
def calculate_total_time(folder_path):
    total_time = 0
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4"):
            video_path = os.path.join(folder_path, filename)
            video_clip = VideoFileClip(video_path)
            # Get video duration in seconds
            video_duration = video_clip.duration
            total_time += video_duration
            video_clip.close()
    return total_time

# Use CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Calculate total time
total_time_seconds = calculate_total_time(folder_path)

# Convert total time to hours, minutes, and seconds
total_hours = int(total_time_seconds // 3600)
total_minutes = int((total_time_seconds % 3600) // 60)
total_seconds = int(total_time_seconds % 60)

print(f"Total time: {total_hours} hours, {total_minutes} minutes, {total_seconds} seconds")


In [ ]:
import os
import torch
from moviepy.editor import VideoFileClip

folder_path = 'E:/Study/VidCropper v 2.2/VidCropper/cropped/Neutral'

# Function to calculate total time in seconds
def calculate_total_time(folder_path):
    total_time = 0
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4"):
            video_path = os.path.join(folder_path, filename)
            video_clip = VideoFileClip(video_path)
            # Get video duration in seconds
            video_duration = video_clip.duration
            total_time += video_duration
            video_clip.close()
    return total_time

# Use CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Calculate total time
total_time_seconds = calculate_total_time(folder_path)

# Convert total time to hours, minutes, and seconds
total_hours = int(total_time_seconds // 3600)
total_minutes = int((total_time_seconds % 3600) // 60)
total_seconds = int(total_time_seconds % 60)

print(f"Total time: {total_hours} hours, {total_minutes} minutes, {total_seconds} seconds")


In [ ]:
import os
import torch
from moviepy.editor import VideoFileClip

folder_path = 'E:/Study/VidCropper v 2.2/VidCropper/cropped/Sad'

# Function to calculate total time in seconds
def calculate_total_time(folder_path):
    total_time = 0
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4"):
            video_path = os.path.join(folder_path, filename)
            video_clip = VideoFileClip(video_path)
            # Get video duration in seconds
            video_duration = video_clip.duration
            total_time += video_duration
            video_clip.close()
    return total_time

# Use CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Calculate total time
total_time_seconds = calculate_total_time(folder_path)

# Convert total time to hours, minutes, and seconds
total_hours = int(total_time_seconds // 3600)
total_minutes = int((total_time_seconds % 3600) // 60)
total_seconds = int(total_time_seconds % 60)

print(f"Total time: {total_hours} hours, {total_minutes} minutes, {total_seconds} seconds")


In [ ]:

folder_path = 'E:/Study/VidCropper v 2.2/VidCropper/cropped/Anger/'  

def calculate_total_time(folder_path):
    total_time = 0
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4"):
            video_path = os.path.join(folder_path, filename)
            video_clip = VideoFileClip(video_path)
            total_time += video_clip.duration
            video_clip.close()
    return total_time

total_time_seconds = calculate_total_time(folder_path)

# Convert total time to hours, minutes, and seconds
total_hours = int(total_time_seconds // 3600)
total_minutes = int((total_time_seconds % 3600) // 60)
total_seconds = int(total_time_seconds % 60)

print(f"Total time: {total_hours} hours, {total_minutes} minutes, {total_seconds} seconds")

In [ ]:
import os
import whisper

model = whisper.load_model("large")
# Get the absolute path to the audio file
audio_file = os.path.abspath("anger.mp3")

result = model.transcribe(audio_file)
print(result["text"])

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)


In [ ]:
#import tensorflow as tf
import os
from moviepy.editor import VideoFileClip
with tf.device('/GPU:0'):
    folder_path = 'C:/Users/Adil Majeed/Downloads/Data/Ghulam/Sad'  
    def calculate_total_time(folder_path):
        total_time = 0
        for filename in os.listdir(folder_path):
            if filename.endswith(".mp4"):
                video_path = os.path.join(folder_path, filename)
                video_clip = VideoFileClip(video_path)
                total_time += video_clip.duration
                video_clip.close()
        return total_time

    total_time_seconds = calculate_total_time(folder_path)

    # Convert total time to hours, minutes, and seconds
    total_hours = int(total_time_seconds // 3600)
    total_minutes = int((total_time_seconds % 3600) // 60)
    total_seconds = int(total_time_seconds % 60)

    print(f"Total time: {total_hours} hours, {total_minutes} minutes, {total_seconds} seconds")
